In [1]:
import onnxruntime as ort
import torch
import numpy as np
from Model import Transformer
from WordPieceTokenizer import WordPieceTokenizer as Tokenizer

saveFilePath = 'saves/'
tokenizer = Tokenizer(f'{saveFilePath}vocab.txt', do_lower_case=False, strip_accents=False, clean_text=True)
VOCAB_SIZE = tokenizer.get_vocab_size()
MAX_SEQUENCE_LENGTH = 128
onnx_filename = f"{saveFilePath}onnx/Sentiment_model.onnx"

# 감정 라벨 매핑
emotion_labels = {
    0: '불안',
    1: '당황',
    2: '분노',
    3: '슬픔',
    4: '중립',
    5: '행복',
    6: '혐오'
}

def get_sentiment_from_onnx(text: str):
    encoded_inputs = tokenizer.encode(
        text,
        max_length=MAX_SEQUENCE_LENGTH,
        add_special_tokens=True,
        padding=True,
        truncation=True
    )
    
    text_input = np.array(encoded_inputs["input_ids"], dtype=np.int64).reshape(1, MAX_SEQUENCE_LENGTH)
    attention_mask_input = np.array(encoded_inputs["attention_mask"], dtype=np.int64).reshape(1, MAX_SEQUENCE_LENGTH)

    try:
        providers = ['CPUExecutionProvider'] 
        ort_session = ort.InferenceSession(onnx_filename, providers=providers)
        
        input_names = [inp.name for inp in ort_session.get_inputs()]
        output_names = [out.name for out in ort_session.get_outputs()]

        ort_inputs = {
            input_names[0]: text_input,
            input_names[1]: attention_mask_input
        }

        ort_output = ort_session.run(output_names, ort_inputs)[0]
        
        predicted_class_id = np.argmax(ort_output, axis=1)[0]
        
        # 숫자 ID를 한글 감정 라벨로 변환
        predicted_emotion = emotion_labels.get(predicted_class_id, "알 수 없음")
        
        return predicted_emotion, ort_output
        
    except Exception as e:
        print(f"ONNX Runtime 추론 중 오류 발생: {e}")
        return None, None

if __name__ == "__main__":
    text_to_analyze = "이 영화 정말 재미있어요!"
    predicted_emotion, logits = get_sentiment_from_onnx(text_to_analyze)
    
    if predicted_emotion is not None:
        print(f"입력 텍스트: \"{text_to_analyze}\"")
        print(f"예측된 감정: {predicted_emotion}")
        print(f"모델 출력 (logits): {logits}")

    text_to_analyze_2 = "정말 지루하고 재미없는 영화였어."
    predicted_emotion_2, logits_2 = get_sentiment_from_onnx(text_to_analyze_2)

    if predicted_emotion_2 is not None:
        print(f"\n입력 텍스트: \"{text_to_analyze_2}\"")
        print(f"예측된 감정: {predicted_emotion_2}")
        print(f"모델 출력 (logits): {logits_2}")

입력 텍스트: "이 영화 정말 재미있어요!"
예측된 감정: 행복
모델 출력 (logits): [[-2.8839788  2.2030983 -1.8115845 -1.3572699  3.097333   3.159402
  -4.191615 ]]

입력 텍스트: "정말 지루하고 재미없는 영화였어."
예측된 감정: 중립
모델 출력 (logits): [[-1.8359216   1.0331647  -0.86719316 -0.7387915   1.2273061   0.96525925
  -2.0229034 ]]


In [2]:
if __name__ == "__main__":
    # 리듬 게임 관련 다양한 감정 테스트 문장 배열
    rhythm_game_sentences = [
        "아 망했다! 콤보 끊겼어! 다시 처음부터 해야 하잖아!",  # 분노/슬픔/불안
        "와 미쳤다! 풀콤보 달성! 내 손이 이렇게 빨랐나?",  # 행복/당황 (기쁨의 당황)
        "이 패턴은 아무리 해도 안 되네... 너무 어려워 포기하고 싶다.",  # 슬픔/불안
        "처음 해보는 곡인데 생각보다 잘 맞아서 놀랐어.",  # 당황/중립
        "정확도를 높이려면 더 연습해야겠어.",  # 중립
        "퍼펙트 놓친 게 너무 짜증나! 딱 하나였는데!",  # 분노/슬픔
        "이 노래는 비트가 너무 좋아서 계속 플레이하게 돼!",  # 행복
        "손가락이 아파서 더 이상 못 하겠네.",  # 슬픔/중립
        "새로운 악보가 나왔는데 너무 기대돼.",  # 행복
        "아니, 분명히 맞게 눌렀는데 미스가 나다니, 말도 안 돼!", # 분노/혐오
        "내 최고 기록을 갱신하다니, 믿을 수 없어!", # 행복/당황
        "이 곡은 정말 아름다운 멜로디를 가지고 있네.", # 중립/행복
        "점수가 너무 낮게 나와서 창피하다.", # 슬픔/불안
        "친구랑 같이 하니까 훨씬 더 재밌어!", # 행복
        "끝까지 집중해야 돼, 조금이라도 방심하면 안 돼!" # 불안/중립
    ]

    print("--- 리듬 게임 관련 감정 분석 테스트 시작 ---")
    for i, sentence in enumerate(rhythm_game_sentences):
        print(f"\n[{i+1}/{len(rhythm_game_sentences)}] 입력 텍스트: \"{sentence}\"")
        predicted_emotion, logits = get_sentiment_from_onnx(sentence)
        
        if predicted_emotion is not None:
            print(f"예측된 감정: {predicted_emotion}")
            # 필요하다면 logits도 출력: print(f"모델 출력 (logits): {logits}")
        else:
            print("감정 분석에 실패했습니다.")
    print("\n--- 리듬 게임 관련 감정 분석 테스트 종료 ---")

--- 리듬 게임 관련 감정 분석 테스트 시작 ---

[1/15] 입력 텍스트: "아 망했다! 콤보 끊겼어! 다시 처음부터 해야 하잖아!"
예측된 감정: 중립

[2/15] 입력 텍스트: "와 미쳤다! 풀콤보 달성! 내 손이 이렇게 빨랐나?"
예측된 감정: 분노

[3/15] 입력 텍스트: "이 패턴은 아무리 해도 안 되네... 너무 어려워 포기하고 싶다."
예측된 감정: 슬픔

[4/15] 입력 텍스트: "처음 해보는 곡인데 생각보다 잘 맞아서 놀랐어."
예측된 감정: 중립

[5/15] 입력 텍스트: "정확도를 높이려면 더 연습해야겠어."
예측된 감정: 중립

[6/15] 입력 텍스트: "퍼펙트 놓친 게 너무 짜증나! 딱 하나였는데!"
예측된 감정: 분노

[7/15] 입력 텍스트: "이 노래는 비트가 너무 좋아서 계속 플레이하게 돼!"
예측된 감정: 행복

[8/15] 입력 텍스트: "손가락이 아파서 더 이상 못 하겠네."
예측된 감정: 슬픔

[9/15] 입력 텍스트: "새로운 악보가 나왔는데 너무 기대돼."
예측된 감정: 혐오

[10/15] 입력 텍스트: "아니, 분명히 맞게 눌렀는데 미스가 나다니, 말도 안 돼!"
예측된 감정: 중립

[11/15] 입력 텍스트: "내 최고 기록을 갱신하다니, 믿을 수 없어!"
예측된 감정: 행복

[12/15] 입력 텍스트: "이 곡은 정말 아름다운 멜로디를 가지고 있네."
예측된 감정: 행복

[13/15] 입력 텍스트: "점수가 너무 낮게 나와서 창피하다."
예측된 감정: 슬픔

[14/15] 입력 텍스트: "친구랑 같이 하니까 훨씬 더 재밌어!"
예측된 감정: 중립

[15/15] 입력 텍스트: "끝까지 집중해야 돼, 조금이라도 방심하면 안 돼!"
예측된 감정: 중립

--- 리듬 게임 관련 감정 분석 테스트 종료 ---


In [1]:
import onnxruntime as ort
import torch
import numpy as np
from Model import Transformer
from WordPieceTokenizer import WordPieceTokenizer as Tokenizer

saveFilePath = 'saves/'
tokenizer = Tokenizer(f'{saveFilePath}vocab.txt', do_lower_case=False, strip_accents=False, clean_text=True)
VOCAB_SIZE = tokenizer.get_vocab_size()
MAX_SEQUENCE_LENGTH = 128
onnx_filename = f"{saveFilePath}onnx/Sentiment_model.onnx"

# 감정 라벨 매핑
emotion_labels = {
    0: '불안',
    1: '당황',
    2: '분노',
    3: '슬픔',
    4: '중립',
    5: '행복',
    6: '혐오'
}

def get_sentiment_from_onnx(text: str):
    encoded_inputs = tokenizer.encode(
        text,
        max_length=MAX_SEQUENCE_LENGTH,
        add_special_tokens=True,
        padding=True,
        truncation=True
    )
    
    text_input = np.array(encoded_inputs["input_ids"], dtype=np.int64).reshape(1, MAX_SEQUENCE_LENGTH)
    attention_mask_input = np.array(encoded_inputs["attention_mask"], dtype=np.int64).reshape(1, MAX_SEQUENCE_LENGTH)

    try:
        providers = ['CPUExecutionProvider'] 
        ort_session = ort.InferenceSession(onnx_filename, providers=providers)
        
        input_names = [inp.name for inp in ort_session.get_inputs()]
        output_names = [out.name for out in ort_session.get_outputs()]

        ort_inputs = {
            input_names[0]: text_input,
            input_names[1]: attention_mask_input
        }

        ort_output = ort_session.run(output_names, ort_inputs)[0]
        
        predicted_class_id = np.argmax(ort_output, axis=1)[0]
        
        # 숫자 ID를 한글 감정 라벨로 변환
        predicted_emotion = emotion_labels.get(predicted_class_id, "알 수 없음")
        
        return predicted_emotion, predicted_class_id, ort_output
        
    except Exception as e:
        print(f"ONNX Runtime 추론 중 오류 발생: {e}")
        return None, None, None

if __name__ == "__main__":
    # 리듬 게임 관련 다양한 감정 테스트 문장 배열
    rhythm_game_sentences = [
        "아 망했다! 콤보 끊겼어! 다시 처음부터 해야 하잖아!",  # 분노/슬픔/불안
        "와 미쳤다! 풀콤보 달성! 내 손이 이렇게 빨랐나?",  # 행복/당황 (기쁨의 당황)
        "이 패턴은 아무리 해도 안 되네... 너무 어려워 포기하고 싶다.",  # 슬픔/불안
        "처음 해보는 곡인데 생각보다 잘 맞아서 놀랐어.",  # 당황/중립
        "정확도를 높이려면 더 연습해야겠어.",  # 중립
        "퍼펙트 놓친 게 너무 짜증나! 딱 하나였는데!",  # 분노/슬픔
        "이 노래는 비트가 너무 좋아서 계속 플레이하게 돼!",  # 행복
        "손가락이 아파서 더 이상 못 하겠네.",  # 슬픔/중립
        "새로운 악보가 나왔는데 너무 기대돼.",  # 행복
        "아니, 분명히 맞게 눌렀는데 미스가 나다니, 말도 안 돼!", # 분노/혐오
        "내 최고 기록을 갱신하다니, 믿을 수 없어!", # 행복/당황
        "이 곡은 정말 아름다운 멜로디를 가지고 있네.", # 중립/행복
        "점수가 너무 낮게 나와서 창피하다.", # 슬픔/불안
        "친구랑 같이 하니까 훨씬 더 재밌어!", # 행복
        "끝까지 집중해야 돼, 조금이라도 방심하면 안 돼!", # 불안/중립
        "진짜 짜증나"
    ]

    print("--- 리듬 게임 관련 감정 분석 테스트 시작 ---")
    for i, sentence in enumerate(rhythm_game_sentences):
        print(f"\n[{i+1}/{len(rhythm_game_sentences)}] 입력 텍스트: \"{sentence}\"")
        predicted_emotion, predicted_id, logits = get_sentiment_from_onnx(sentence)
        
        if predicted_emotion is not None:
            print(f"예측된 감정 (한글): {predicted_emotion}")
            print(f"예측된 감정 (ID): {predicted_id}")
            print(f"모델 원본 출력 (logits): {logits}")
        else:
            print("감정 분석에 실패했습니다.")
    print("\n--- 리듬 게임 관련 감정 분석 테스트 종료 ---")

--- 리듬 게임 관련 감정 분석 테스트 시작 ---

[1/16] 입력 텍스트: "아 망했다! 콤보 끊겼어! 다시 처음부터 해야 하잖아!"
예측된 감정 (한글): 중립
예측된 감정 (ID): 4
모델 원본 출력 (logits): [[-3.610796   -1.2480685   2.241982    0.72675717  2.5493803  -1.9717649
  -3.3849218 ]]

[2/16] 입력 텍스트: "와 미쳤다! 풀콤보 달성! 내 손이 이렇게 빨랐나?"
예측된 감정 (한글): 분노
예측된 감정 (ID): 2
모델 원본 출력 (logits): [[-2.404556  -2.4789922  3.8352675 -1.834424   0.7831502 -1.2238318
   2.2334993]]

[3/16] 입력 텍스트: "이 패턴은 아무리 해도 안 되네... 너무 어려워 포기하고 싶다."
예측된 감정 (한글): 슬픔
예측된 감정 (ID): 3
모델 원본 출력 (logits): [[ 0.26082417 -2.5314198   1.338215    2.1571324  -0.9929757  -3.138421
  -0.47905505]]

[4/16] 입력 텍스트: "처음 해보는 곡인데 생각보다 잘 맞아서 놀랐어."
예측된 감정 (한글): 중립
예측된 감정 (ID): 4
모델 원본 출력 (logits): [[ 0.31053203  0.10183488 -2.5870667  -0.6155038   3.9963355   0.02883576
  -1.9727983 ]]

[5/16] 입력 텍스트: "정확도를 높이려면 더 연습해야겠어."
예측된 감정 (한글): 중립
예측된 감정 (ID): 4
모델 원본 출력 (logits): [[ 0.00547738 -2.596329    0.19450162 -0.28711718  3.1669564  -2.174234
   0.3537476 ]]

[6/16] 입력 텍스트: "퍼펙트 놓친 게 너무 짜증나! 딱 하나였는데!"
예측된 